In [1]:
from src.backtesting_program import BacktestingPorgram
from src.checker import Checker
import pandas as pd
from config import settings
from src.service import Service

In [2]:
# data = pd.read_csv("data.csv")
service = Service()
data = service.take_polygon_gold_historical_data(
  from_="2025-01-01",
  to="2025-03-01",
  limit=50000,
  candle_size=15,
  tz_convert='America/New_York'
  )

In [3]:
data

,Time,Open,High,Low,Close,Volume
0,2025-01-01 18:00:00-0500,2624.80,2625.01,2623.90,2624.26,38
1,2025-01-01 18:15:00-0500,2624.31,2624.39,2623.18,2623.57,40
2,2025-01-01 18:30:00-0500,2623.54,2623.58,2621.40,2622.13,45
3,2025-01-01 18:45:00-0500,2621.46,2623.98,2620.70,2623.68,40
4,2025-01-01 19:00:00-0500,2623.52,2625.96,2622.00,2625.64,40
...,...,...,...,...,...,...
3882,2025-02-28 15:45:00-0500,2849.79,2854.37,2849.41,2854.37,38
3883,2025-02-28 16:00:00-0500,2854.56,2854.86,2852.78,2854.46,38
3884,2025-02-28 16:15:00-0500,2854.74,2857.43,2854.42,2856.36,39
3885,2025-02-28 16:30:00-0500,2856.78,2858.44,2856.28,2857.49,39


In [4]:
for el in data.itertuples():
  print(type(el.Time))
  break

<class 'str'>


In [5]:
settings.LOGS_FILE_PATH = "output/logs.csv"
settings.EXECUTED_TRADES_FILE_PATH = "output/executed_trades.json"

checker = Checker(
  active_checkers={
    "when_trade_is_opened" : {
      "global": [
        "end_time_checker"
      ],
      "to_close_sell": [
        "updating_sell_stop_loss_checker",
        "to_close_sell_trade_1_checker",
        "to_close_sell_trade_2_checker",
        "to_close_sell_trade_3_checker"
      ],
      "to_close_buy": [
        "updating_buy_stop_loss_checker",
        "to_close_buy_trade_1_checker",
        "to_close_buy_trade_2_checker",
        "to_close_buy_trade_3_checker"
      ]
    },
    "when_trade_is_not_opened" : {
      "global": [
      "start_time_checker",
      "no_more_trades_time_checker",
      ],
      # if checker returns "SKIP", program skips all "to_open_sell" checkers and goes to "to_open_buy" checkers
      "to_open_sell": [
        "if_sell_was_executed_once_this_day",
        "to_sell_order_1_checker",
      ],
      "to_open_buy": [
      "if_buy_was_executed_once_this_day",
      "to_buy_order_1_checker",
      ]
    }
  },
  data=data,
  params={"start_time": "06:00", "end_time": "23:00", "no_more_trades_time": "21:30"},
)

program = BacktestingPorgram(
  historical_data=data,
  checker=checker,
)

program.start()

In [6]:
program.summary

,Opened Trades,Win Ratio,Average R:R,P/L,Consecutive Losses,Losses Quantity,BE Quantity,Win Quantity
0,49,44.89795918367347%,1:2.86,4.06%,4,27,0,22
